# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [1]:
from sklearn import model_selection, datasets, linear_model, metrics
import numpy as np
import pandas as pd

### Генерация датасета

In [2]:
iris = datasets.load_iris()

In [3]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(
    iris.data, iris.target, test_size=0.3,random_state=0)

### Задание модели

In [4]:
classifier = linear_model.SGDClassifier(random_state=0)

### Генерация сетки

In [5]:
classifier.get_params().keys()

dict_keys(['learning_rate', 'n_jobs', 'average', 'power_t', 'eta0', 'fit_intercept', 'class_weight', 'shuffle', 'epsilon', 'l1_ratio', 'alpha', 'warm_start', 'n_iter', 'penalty', 'random_state', 'verbose', 'loss'])

In [6]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'n_iter' : range(5, 10),
    'alpha' : np.linspace(0.0001, 0.001, num=5),
}

In [7]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

### Подбор параметров и оценка качества

#### Grid search

In [8]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring='accuracy', cv=cv)

In [9]:
%%time
grid_cv.fit(train_data, train_labels)

CPU times: user 7.51 s, sys: 92.2 ms, total: 7.61 s
Wall time: 8.07 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
       error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'alpha': array([ 0.0001 ,  0.00032,  0.00055,  0.00078,  0.001  ]), 'n_iter': range(5, 10), 'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [10]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.00032499999999999999, average=False, class_weight=None,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=9, n_jobs=1,
       penalty='l1', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [11]:
print('Best accuracy score:', grid_cv.best_score_)
print('Best parameters:\n', grid_cv.best_params_)

Best accuracy score: 0.895238095238
Best parameters:
 {'penalty': 'l1', 'alpha': 0.00032499999999999999, 'n_iter': 9, 'loss': 'hinge'}


In [12]:
print('Примеры получившегося отчета:')
for i in range(5):
    print('\nAccuracy: ', grid_cv.cv_results_['mean_test_score'][i])
    print('Parameters: ', grid_cv.cv_results_['params'][i])

Примеры получившегося отчета:

Accuracy:  0.757142857143
Parameters:  {'penalty': 'l1', 'alpha': 0.0001, 'n_iter': 5, 'loss': 'hinge'}

Accuracy:  0.666666666667
Parameters:  {'penalty': 'l2', 'alpha': 0.0001, 'n_iter': 5, 'loss': 'hinge'}

Accuracy:  0.614285714286
Parameters:  {'penalty': 'l1', 'alpha': 0.0001, 'n_iter': 6, 'loss': 'hinge'}

Accuracy:  0.685714285714
Parameters:  {'penalty': 'l2', 'alpha': 0.0001, 'n_iter': 6, 'loss': 'hinge'}

Accuracy:  0.795238095238
Parameters:  {'penalty': 'l1', 'alpha': 0.0001, 'n_iter': 7, 'loss': 'hinge'}


#### Randomized grid search

In [13]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, 
                                                        scoring ='accuracy', cv=cv, n_iter=20)

In [14]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

CPU times: user 767 ms, sys: 14.1 ms, total: 781 ms
Wall time: 952 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
          error_score='raise',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False),
          fit_params={}, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'penalty': ['l1', 'l2'], 'alpha': array([ 0.0001 ,  0.00032,  0.00055,  0.00078,  0.001  ]), 'n_iter': range(5, 10), 'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='accuracy', verbose=0)

In [15]:
print('Best accuracy: ', randomized_grid_cv.best_score_)
print('Best parameters: ', randomized_grid_cv.best_params_)

Best accuracy:  0.885714285714
Best parameters:  {'penalty': 'l1', 'alpha': 0.00032499999999999999, 'loss': 'squared_hinge', 'n_iter': 9}
